# Universe 模块教程

本教程介绍 open-xquant 的标的池构建模块：如何定义和筛选**"值得关注"的标的**。

Universe 模块提供两种构建方式：
- **StaticUniverse** — 手动指定固定标的列表
- **FilterUniverse** — 基于量价数据的规则筛选

核心设计原则：**Universe 是 mktdata 的消费者，不是数据的加载者**——与 Indicator/Signal 同构，接收共享宽表作为输入。

## 1. 安装依赖

Universe 模块是 open-xquant 核心功能，无需额外依赖。如需结合 Data 模块下载真实数据：

```bash
pip install open-xquant[yfinance]
```

---
## 2. StaticUniverse — 手动指定标的池

最简单的 Universe：直接给出标的列表。适用于你已经通过研究确定了目标标的的场景。

In [ ]:
from oxq.universe import StaticUniverse

# 全球宏观轮动：沪深300 ETF、纳斯达克100 ETF、黄金 ETF
universe = StaticUniverse(
    symbols=["510300.SS", "QQQ", "GLD"],
    name="global-macro-etf",
)

snapshot = universe.get_universe(as_of_date="2024-12-31")
print(f"标的列表: {snapshot.symbols}")
print(f"来源标识: {snapshot.source}")
print(f"截面日期: {snapshot.as_of_date}")

`get_universe(as_of_date)` 返回一个不可变的 `UniverseSnapshot`，包含：
- **as_of_date** — 截面日期
- **symbols** — 标的元组（有序、不可变）
- **source** — 来源标识，便于追溯
- **metadata** — 扩展信息（预留）

StaticUniverse 不需要任何市场数据，`get_universe()` 直接返回你指定的列表。它的典型用途是把研究结论固化为代码：

In [ ]:
# 美股科技龙头
tech_universe = StaticUniverse(symbols=["AAPL", "MSFT", "GOOGL", "AMZN", "META"])

snap = tech_universe.get_universe(as_of_date="2024-12-31")
print(f"标的数量: {len(snap.symbols)}")
print(f"来源: {snap.source}")

---
## 3. 准备数据 — 下载候选池行情

FilterUniverse 需要市场数据来执行筛选。我们先用 Data 模块下载一组 ETF 的行情数据，构建 `mktdata` 字典。

In [ ]:
from oxq.data import YFinanceDownloader, LocalMarketDataProvider

downloader = YFinanceDownloader()

# 候选 ETF 池：美股大盘、科技、黄金、债券、新兴市场、房地产、能源
candidates = ["SPY", "QQQ", "GLD", "TLT", "EEM", "VNQ", "XLE"]

paths = downloader.download_many(candidates, start="2024-01-01", end="2024-12-31")
for symbol, path in paths.items():
    print(f"{symbol}: {path}")

用 `LocalMarketDataProvider` 读取数据，构建 FilterUniverse 需要的 `mktdata` 字典。

新版 API 中，`mktdata` 在构造 FilterUniverse 时注入，而不是在 `get_universe()` 调用时传入：

In [ ]:
provider = LocalMarketDataProvider()

# 构建 mktdata: {symbol: DataFrame}
mktdata = {
    symbol: provider.get_bars(symbol, "2024-01-01", "2024-12-31")
    for symbol in candidates
}

for symbol, df in mktdata.items():
    latest = df.iloc[-1]
    print(f"{symbol}: close={latest['close']:.2f}, volume={int(latest['volume']):,}")

---
## 4. FilterUniverse — 量价规则筛选

FilterUniverse 在候选池上施加筛选条件。`mktdata` 在构造时注入，`get_universe(as_of_date)` 使用 `as_of_date` 切片数据，取截面日期当天或之前最近一行来评估每个标的。

In [ ]:
from oxq.universe import FilterUniverse, Filter

universe = FilterUniverse(
    base=candidates,
    filters=[
        Filter(column="volume", op=">=", value=5_000_000),   # 日成交量 >= 500万股
        Filter(column="close", op=">=", value=50.0),         # 收盘价 >= 50 美元
    ],
    mktdata=mktdata,
    name="liquid-etf",
)

snapshot = universe.get_universe(as_of_date="2024-12-31")
print(f"候选池: {len(universe.base)} 个标的")
print(f"筛选后: {snapshot.metadata['filtered_count']} 个标的")
print(f"通过筛选: {snapshot.symbols}")
print(f"来源: {snapshot.source}")
print(f"截面日期: {snapshot.as_of_date}")

`Filter` 是一个不可变的筛选条件，由三个字段组成：
- **column** — mktdata 中的列名（如 `volume`、`close`）
- **op** — 比较运算符（`>=`、`<=`、`>`、`<`、`==`、`!=`）
- **value** — 阈值

多个 Filter 之间是 **AND 关系**——标的必须同时满足所有条件才会被保留。

调整筛选条件，观察结果变化：

In [ ]:
# 宽松条件：仅要求有基本流动性
loose = FilterUniverse(
    base=candidates,
    filters=[Filter(column="volume", op=">=", value=1_000_000)],
    mktdata=mktdata,
)
snap_loose = loose.get_universe(as_of_date="2024-12-31")

# 严格条件：高流动性 + 高价
strict = FilterUniverse(
    base=candidates,
    filters=[
        Filter(column="volume", op=">=", value=20_000_000),
        Filter(column="close", op=">=", value=100.0),
    ],
    mktdata=mktdata,
)
snap_strict = strict.get_universe(as_of_date="2024-12-31")

print(f"宽松筛选: {snap_loose.symbols}")
print(f"严格筛选: {snap_strict.symbols}")

---
## 5. 组合使用 — 完整管线

将 Data 下载 → 构建 mktdata → FilterUniverse 筛选串联为一段连贯代码：

In [ ]:
from oxq.data import YFinanceDownloader, LocalMarketDataProvider
from oxq.universe import FilterUniverse, Filter

# 1. 下载数据
downloader = YFinanceDownloader()
pool = ["SPY", "QQQ", "GLD", "TLT", "EEM", "VNQ", "XLE"]
downloader.download_many(pool, start="2024-01-01", end="2024-12-31")

# 2. 构建 mktdata
provider = LocalMarketDataProvider()
mktdata = {
    s: provider.get_bars(s, "2024-01-01", "2024-12-31")
    for s in pool
}

# 3. 筛选
universe = FilterUniverse(
    base=pool,
    filters=[
        Filter(column="volume", op=">=", value=5_000_000),
        Filter(column="close", op=">=", value=50.0),
    ],
    mktdata=mktdata,
    name="liquid-etf",
)
result = universe.get_universe(as_of_date="2024-12-31")

print(f"最终标的池: {result.symbols}")
print(f"筛选比例: {result.metadata['filtered_count']}/{result.metadata['base_count']}")

---
## 小结

本教程覆盖了 Universe 模块的核心用法：

| 组件 | 职责 |
|------|------|
| `StaticUniverse` | 手动指定固定标的列表 |
| `FilterUniverse` | 基于 mktdata 量价数据的规则筛选 |
| `Filter` | 单条筛选条件（column + op + value） |
| `UniverseSnapshot` | 筛选结果的不可变快照 |

**核心 API**：
- `get_universe(as_of_date)` — 获取指定截面日期的标的快照
- `get_history(start, end)` — 获取日期范围内的历史快照

**核心设计原则**：
- Universe 是 mktdata 的消费者，不负责加载数据
- `mktdata` 在构造时注入，`get_universe()` 通过 `as_of_date` 切片
- 所有构建结果都是不可变快照（`frozen=True`）
- StaticUniverse 适合固化研究结论，FilterUniverse 适合规则驱动的动态筛选